In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

class StockDataset(Dataset):
    def __init__(self, data: pd.DataFrame, window_size: int = 5, stride: int = 1):
        """
        Args:
            data: DataFrame with columns [Open, Close, High, Low, return_ratio, 
                  %change_open, %change_high, %change_low, MA5, MA10, MA15, MA20, MA25, MA30, 
                  up_down, sector]
            window_size: Number of days in a week (5)
            stride: Stride for sliding window
        """
        self.data = data
        self.window_size = window_size
        self.stride = stride
        
        # Create weekly windows
        self.windows = []
        for i in range(0, len(data) - window_size + 1, stride):
            self.windows.append(data.iloc[i:i+window_size])
            
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        window = self.windows[idx]
        features = window[['Open', 'Close', 'High', 'Low', 'Return_Ratio',
                        'Percent_Change_Open', 'Percentage_Change_High', 'Percentage_Change_Low',
                        'MA_5', 'MA_10', 'MA_15', 'MA_20', 'MA_25', 'MA_30','Sector_ecnoded']].values
        movement = window['up_down'].values[-1]  # Last day's movement
        return_ratio = window['Return_Ratio'].values[-1]  # Last day's return ratio
        
        return {
            'features': torch.FloatTensor(features),
            'movement': torch.FloatTensor([movement]),
            'return_ratio': torch.FloatTensor([return_ratio])
        }

class AttentiveGRU(nn.Module):
    def __init__(self, input_dim: int = 15, hidden_dim: int = 16):
        super(AttentiveGRU, self).__init__()
        self.gru1 = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
        )
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        output, _ = self.gru1(x)  # output shape: (batch_size, seq_len, hidden_dim)
        output, _ = self.gru2(output)
        # Compute attention weights
        attention_weights = self.attention(output)  # shape: (batch_size, seq_len, 1)
        attention_weights = F.softmax(attention_weights, dim=1)
        
        # Apply attention
        a_i = torch.sum(output * attention_weights, dim=1)  # shape: (batch_size, hidden_dim)
        return a_i

class IntraSectorGAT(nn.Module):
    def __init__(self, input_dim: int = 16, output_dim: int = 16):
        super(IntraSectorGAT, self).__init__()
        self.W = nn.Linear(input_dim, output_dim, bias=False)
        self.a = nn.Parameter(torch.empty(size=(2*output_dim, 1)))
        nn.init.xavier_uniform_(self.a.data)
        
    def forward(self, x, adj):
        # x shape: (num_nodes, input_dim)
        # adj shape: (num_nodes, num_nodes)
        
        Wx = self.W(x)  # shape: (num_nodes, output_dim)
        
        # Compute attention coefficients
        a_input = torch.cat([Wx.repeat(1, Wx.size(0)).view(Wx.size(0) * Wx.size(0), -1),
                            Wx.repeat(Wx.size(0), 1)], dim=1)
        a_input = a_input.view(Wx.size(0), Wx.size(0), 2 * Wx.size(1))
        
        e = torch.matmul(a_input, self.a).squeeze(2)
        attention = F.softmax(F.leaky_relu(e) * adj, dim=1)
        
        # Apply attention to neighbors
        h = torch.matmul(attention, Wx)
        return h

class FinGAT(nn.Module):
    def __init__(self, input_dim: int = 15, hidden_dim: int = 16):
        super(FinGAT, self).__init__()
        self.attentive_gru = AttentiveGRU(input_dim, hidden_dim)
        self.intra_sector_gat = IntraSectorGAT(hidden_dim, hidden_dim)
        self.inter_sector_gat = IntraSectorGAT(hidden_dim, hidden_dim)
        
        # Prediction heads
        self.movement_head = nn.Linear(hidden_dim * 3, 1)  # Binary classification
        self.return_head = nn.Linear(hidden_dim * 3, 1)  # Return ratio prediction
        
    def forward(self, x, intra_adj, inter_adj):
        # Short-term sequential learning
        gru_embed = self.attentive_gru(x)
        
        # Intra-sector modeling
        intra_embed = self.intra_sector_gat(gru_embed, intra_adj)
        
        # Inter-sector modeling
        inter_embed = self.inter_sector_gat(intra_embed, inter_adj)
        
        # Combine embeddings
        combined = torch.cat([gru_embed, intra_embed, inter_embed], dim=1)
        
        # Multi-task predictions
        movement_pred = torch.sigmoid(self.movement_head(combined))
        return_pred = self.return_head(combined)
        
        return movement_pred, return_pred

class PairwiseRankingLoss(nn.Module):
    def __init__(self, margin: float = 0.1):
        super(PairwiseRankingLoss, self).__init__()
        self.margin = margin
        
    def forward(self, predictions, targets):
        # Compute pairwise differences
        diff_pred = predictions.unsqueeze(0) - predictions.unsqueeze(1)
        diff_target = targets.unsqueeze(0) - targets.unsqueeze(1)
        
        # Convert target differences to binary labels
        target_labels = (diff_target > 0).float()
        
        # Compute ranking loss
        loss = F.relu(self.margin - diff_pred * target_labels)
        return loss.mean()

def train_fingat(model: FinGAT, train_loader: DataLoader, val_loader: DataLoader, learning_rate: float = 0.001, delta: float = 0.01, epochs: int = 50):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    ranking_loss = PairwiseRankingLoss()
    movement_loss = nn.BCELoss()
    
    best_mrr = 0
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch in train_loader:
            optimizer.zero_grad()
            
            movement_pred, return_pred = model(batch['features'],
                                             batch['intra_adj'],
                                             batch['inter_adj'])
            
            # Compute losses
            rank_loss = ranking_loss(return_pred, batch['return_ratio'])
            mov_loss = movement_loss(movement_pred, batch['movement'])
            
            # Combined loss with balancing parameter
            loss = rank_loss + delta * mov_loss
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validation
        model.eval()
        val_mrr = evaluate_mrr(model, val_loader, k=10)
        
        if val_mrr > best_mrr:
            best_mrr = val_mrr
            torch.save(model.state_dict(), 'best_fingat_model.pt')
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation MRR@10: {val_mrr:.4f}')

def evaluate_mrr(model: FinGAT, data_loader: DataLoader, k: int = 10):
    model.eval()
    reciprocal_ranks = []
    
    with torch.no_grad():
        for batch in data_loader:
            _, return_pred = model(batch['features'],
                                 batch['intra_adj'],
                                 batch['inter_adj'])
            
            # Get rankings
            pred_ranks = torch.argsort(return_pred, descending=True)
            true_ranks = torch.argsort(batch['return_ratio'], descending=True)
            
            # Compute MRR@k
            for i in range(min(k, len(pred_ranks))):
                if pred_ranks[i] in true_ranks[:k]:
                    reciprocal_ranks.append(1.0 / (i + 1))
                    break
    
    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0

# Hyperparameter configurations
hyperparameters = {
    'window_sizes': [5, 10, 15, 20],  # Corresponding to 1,2,3,4 weeks
    'hidden_dims': [8, 16, 32, 64],
    'deltas': [0, 0.0001, 0.001, 0.01, 0.1, 1],
    'learning_rates': [0.0005, 0.001, 0.005],
    'batch_sizes': [32, 64, 128]
}

In [17]:
import torch
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple

def create_sector_mappings(df: pd.DataFrame) -> Tuple[Dict[str, int], Dict[int, List[str]]]:
    """
    Create mappings between sectors and stocks
    
    Args:
        df: DataFrame with 'Symbol' and 'Sector' columns
        
    Returns:
        sector_to_id: Dictionary mapping sector names to sector IDs
        sector_stocks: Dictionary mapping sector IDs to list of stock symbols in that sector
    """
    # Create sector to ID mapping
    unique_sectors = df['Sector'].unique()
    sector_to_id = {sector: idx for idx, sector in enumerate(unique_sectors)}
    
    # Create sector to stocks mapping
    sector_stocks = {}
    for sector_id in range(len(unique_sectors)):
        sector_name = unique_sectors[sector_id]
        stocks_in_sector = df[df['Sector'] == sector_name]['Symbol'].tolist()
        sector_stocks[sector_id] = stocks_in_sector
        
    return sector_to_id, sector_stocks

def create_stock_to_sector_mapping(df: pd.DataFrame) -> Dict[str, int]:
    """
    Create mapping from stock symbols to their sector IDs
    
    Args:
        df: DataFrame with 'Symbol' and 'Sector' columns
        
    Returns:
        Dictionary mapping stock symbols to their sector IDs
    """
    sector_to_id, _ = create_sector_mappings(df)
    return {row['Symbol']: sector_to_id[row['Sector']] 
            for _, row in df.iterrows()}

def create_intra_sector_adjacency(df: pd.DataFrame) -> torch.Tensor:
    """
    Create intra-sector adjacency matrix where stocks in the same sector are connected
    
    Args:
        df: DataFrame with 'Symbol' and 'Sector' columns
        
    Returns:
        Adjacency matrix as torch tensor
    """
    num_stocks = len(df)
    adjacency = torch.zeros((num_stocks, num_stocks))
    
    # Get stock to sector mapping
    stock_to_sector = create_stock_to_sector_mapping(df)
    stock_to_idx = {symbol: idx for idx, symbol in enumerate(df['Symbol'])}
    
    # Connect stocks in the same sector
    for i, stock1 in enumerate(df['Symbol']):
        for j, stock2 in enumerate(df['Symbol']):
            if i != j and stock_to_sector[stock1] == stock_to_sector[stock2]:
                adjacency[i, j] = 1.0
                
    return adjacency

def create_inter_sector_adjacency(df: pd.DataFrame) -> torch.Tensor:
    """
    Create inter-sector adjacency matrix where sectors are nodes
    
    Args:
        df: DataFrame with 'Symbol' and 'Sector' columns
        
    Returns:
        Sector-level adjacency matrix as torch tensor
    """
    sector_to_id, _ = create_sector_mappings(df)
    num_sectors = len(sector_to_id)
    
    # Create fully connected sector graph (excluding self-loops)
    adjacency = torch.ones((num_sectors, num_sectors)) - torch.eye(num_sectors)
    
    return adjacency

def create_sector_to_stock_adjacency(df: pd.DataFrame) -> torch.Tensor:
    """
    Create adjacency matrix mapping sectors to their stocks
    
    Args:
        df: DataFrame with 'Symbol' and 'Sector' columns
        
    Returns:
        Bipartite adjacency matrix as torch tensor
    """
    sector_to_id, _ = create_sector_mappings(df)
    num_sectors = len(sector_to_id)
    num_stocks = len(df)
    
    adjacency = torch.zeros((num_sectors, num_stocks))
    stock_to_sector = create_stock_to_sector_mapping(df)
    
    for stock_idx, symbol in enumerate(df['Symbol']):
        sector_id = stock_to_sector[symbol]
        adjacency[sector_id, stock_idx] = 1.0
        
    return adjacency

def normalize_adjacency(adjacency: torch.Tensor) -> torch.Tensor:
    """
    Normalize adjacency matrix using symmetric normalization
    
    Args:
        adjacency: Input adjacency matrix
        
    Returns:
        Normalized adjacency matrix
    """
    # Add self-loops
    adjacency = adjacency + torch.eye(adjacency.shape[0])
    
    # Calculate degree matrix
    degree = torch.sum(adjacency, dim=1)
    degree_sqrt_inv = torch.diag(torch.pow(degree, -0.5))
    
    # Symmetric normalization
    normalized = torch.mm(torch.mm(degree_sqrt_inv, adjacency), degree_sqrt_inv)
    
    return normalized

# Example usage:
def prepare_adjacency_matrices(stock_data: pd.DataFrame) -> Dict[str, torch.Tensor]:
    """
    Prepare all necessary adjacency matrices for FinGAT
    
    Args:
        stock_data: DataFrame with stock information including symbols and sectors
        
    Returns:
        Dictionary containing all adjacency matrices
    """
    # Create basic adjacency matrices
    intra_sector_adj = create_intra_sector_adjacency(stock_data)
    inter_sector_adj = create_inter_sector_adjacency(stock_data)
    sector_to_stock_adj = create_sector_to_stock_adjacency(stock_data)
    
    # Normalize adjacency matrices
    normalized_intra_adj = normalize_adjacency(intra_sector_adj)
    normalized_inter_adj = normalize_adjacency(inter_sector_adj)
    
    return {
        'intra_sector': normalized_intra_adj,
        'inter_sector': normalized_inter_adj,
        'sector_to_stock': sector_to_stock_adj,
        'raw_intra_sector': intra_sector_adj,
        'raw_inter_sector': inter_sector_adj
    }

# # Example of how to use:
# if __name__ == "__main__":
#     # Sample data
#     sample_data = pd.DataFrame({
#         'Symbol': ['AAPL', 'MSFT', 'JPM', 'BAC', 'GOOGL'],
#         'Sector': ['Technology', 'Technology', 'Finance', 'Finance', 'Technology']
#     })

In [18]:
import os
import pandas as pd

data_sectors={
    'Symbols' :[],
    'Sector' : []
}
file_path = r'./Data_is_here'
for file in os.listdir(file_path):
    df = pd.read_csv(file_path+"/"+file)
    # print(df.info)
    sector = df['Sector'][5]
    data_sectors['Symbols'].append(file[:-9])
    data_sectors['Sector'].append(sector)


In [26]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from typing import Dict, List, Tuple

def initialize_fingat(config: dict = None):
    """
    Initialize FinGAT model with configuration
    
    Args:
        config: Dictionary containing model hyperparameters
    """
    if config is None:
        config = {
            'input_dim': 15,          # Number of features
            'hidden_dim': 16,         # Hidden dimension size
            'num_weeks': 1,           # Number of weeks (1,2,3,4)
            'window_size': 5,         # Days per week
            'batch_size': 64,
            'learning_rate': 0.001,
            'delta': 0.01,            # Balance parameter for multi-task learning
        }
    
    # Initialize model
    model = FinGAT(
        input_dim=config['input_dim'],
        hidden_dim=config['hidden_dim']
    )
    
    # Initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    
    # Initialize loss functions
    ranking_loss = PairwiseRankingLoss()
    movement_loss = torch.nn.BCELoss()
    
    return model, optimizer, ranking_loss, movement_loss

def create_dataloaders(stock_data: Dict[str, pd.DataFrame],
                      adj_matrices: Dict[str, torch.Tensor],
                      config: dict) -> Tuple[DataLoader, DataLoader, DataLoader]:
    """
    Create train, validation, and test dataloaders
    """
    # Calculate split indices
    total_days = len(next(iter(stock_data.values())))
    train_days = 400
    val_days = 160
    
    # Split data
    train_data = {symbol: df.iloc[:train_days] for symbol, df in stock_data.items()}
    val_data = {symbol: df.iloc[train_days:train_days+val_days] for symbol, df in stock_data.items()}
    test_data = {symbol: df.iloc[train_days+val_days:] for symbol, df in stock_data.items()}
    
    # Create datasets
    train_dataset = StockDataset(train_data, adj_matrices, window_size=config['window_size'])
    val_dataset = StockDataset(val_data, adj_matrices, window_size=config['window_size'])
    test_dataset = StockDataset(test_data, adj_matrices, window_size=config['window_size'])
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])
    
    return train_loader, val_loader, test_loader

def load_stock_data():
    
    file_path = './Preprocessed_data'
    stock_data = {}
    for file in os.listdir(file_path):
        symbol = file.split('.')[0]
        df = pd.read_csv(os.path.join(file_path, file))
        stock_data[symbol] = df
    return stock_data

# Example usage
def setup_and_train():
    """Complete setup and training pipeline"""
    
    # 1. Define configuration
    config = {
        'input_dim': 15,
        'hidden_dim': 16,
        'num_weeks': 1,
        'window_size': 5,
        'batch_size': 64,
        'learning_rate': 0.001,
        'delta': 0.01,
        'epochs': 50
    }
    
    # 2. Load and prepare data
    stock_data = load_stock_data()
    # stock_data = 
    features_dict = data_sectors
    adj_matrices = prepare_adjacency_matrices(stock_data)
    
    # 3. Initialize model and components
    model, optimizer, ranking_loss, movement_loss = initialize_fingat(config)
    
    # 4. Create dataloaders
    train_loader, val_loader, test_loader = create_dataloaders(
        features_dict, adj_matrices, config
    )
    
    # 5. Training loop
    best_val_mrr = 0
    for epoch in range(config['epochs']):
        # Training
        model.train()
        total_loss = 0
        
        for batch in train_loader:
            optimizer.zero_grad()
            
            # Forward pass
            movement_pred, return_pred = model(
                batch['features'],
                batch['intra_adj'],
                batch['inter_adj']
            )
            
            # Calculate losses
            rank_loss = ranking_loss(return_pred, batch['return_ratio'])
            mov_loss = movement_loss(movement_pred, batch['movement'])
            
            # Combined loss
            loss = rank_loss + config['delta'] * mov_loss
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Validation
        val_mrr = evaluate_mrr(model, val_loader)
        
        # Save best model
        if val_mrr > best_val_mrr:
            best_val_mrr = val_mrr
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_mrr': val_mrr,
                'config': config
            }, 'best_fingat_model.pt')
        
        print(f'Epoch {epoch+1}/{config["epochs"]}:')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation MRR: {val_mrr:.4f}')
    
    return model

# Quick initialization for testing
def quick_test():
    """Initialize model with default settings for testing"""
    config = {
        'input_dim': 15,
        'hidden_dim': 16,
        'num_weeks': 1,
        'window_size': 5,
        'batch_size': 64,
        'learning_rate': 0.001,
        'delta': 0.01
    }
    
    model, optimizer, ranking_loss, movement_loss = initialize_fingat(config)
    print("Model initialized with structure:")
    print(model)
    
    # Test with random data
    batch_size = 32
    window_size = 5
    num_features = 15
    test_features = torch.randn(batch_size, window_size, num_features)
    test_intra_adj = torch.ones(batch_size, batch_size)
    test_inter_adj = torch.ones(32, 32)  # Assuming 10 sectors
    
    movement_pred, return_pred = model(test_features, test_intra_adj, test_inter_adj)
    print("\nTest forward pass successful!")
    print(f"Movement predictions shape: {movement_pred.shape}")
    print(f"Return predictions shape: {return_pred.shape}")

if __name__ == "__main__":
    quick_test()

Model initialized with structure:
FinGAT(
  (attentive_gru): AttentiveGRU(
    (gru): GRU(15, 16, batch_first=True)
    (attention): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): Tanh()
      (2): Linear(in_features=16, out_features=1, bias=True)
    )
  )
  (intra_sector_gat): IntraSectorGAT(
    (W): Linear(in_features=16, out_features=16, bias=False)
  )
  (inter_sector_gat): IntraSectorGAT(
    (W): Linear(in_features=16, out_features=16, bias=False)
  )
  (movement_head): Linear(in_features=48, out_features=1, bias=True)
  (return_head): Linear(in_features=48, out_features=1, bias=True)
)

Test forward pass successful!
Movement predictions shape: torch.Size([32, 1])
Return predictions shape: torch.Size([32, 1])
